<a href="https://colab.research.google.com/github/Gucchyon/RandomForestSHAP/blob/main/BiocharRandomForestSHAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#データセットの準備

In [ ]:
# ============= １．データセットの準備 =============

# 必要なライブラリのインストール
!pip install scikit-learn xgboost shap pandas numpy matplotlib seaborn openpyxl tensorflow scikeras

# ライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import warnings
import os
from google.colab import files
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.cross_decomposition import PLSRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.callbacks import EarlyStopping

# scikerasをインポート（TensorFlowの新しいバージョンではtensorflow.keras.wrappers.scikit_learnが廃止されたため）
from scikeras.wrappers import KerasRegressor

# Create main folders for each section
sections = [
    "section1_data_preparation",
    "section2_model_comparison",
    "section3_final_model",
    "section4_predictions_analysis",
    "section5_shap_analysis",
    "section6_interaction_analysis",
    "section7_feature_pdp"
]

# Create each folder
for section in sections:
    if not os.path.exists(section):
        os.makedirs(section)
        print(f"Created folder: {section}")

# Create subfolder for histograms in section 1
histograms_dir = os.path.join("section1_data_preparation", "histograms")
if not os.path.exists(histograms_dir):
    os.makedirs(histograms_dir)
    print(f"Created folder: {histograms_dir}")

# Create subfolder for extreme values analysis in section 4
extremes_dir = os.path.join("section4_predictions_analysis", "extreme_values")
if not os.path.exists(extremes_dir):
    os.makedirs(extremes_dir)
    print(f"Created folder: {extremes_dir}")

# ランダムシードの設定
np.random.seed(42)

# 警告を無視
warnings.filterwarnings('ignore')

# BiocharDS_V1.0.xlsxファイルが存在するか確認
try:
    # 既存ファイルを確認
    biochar_data = pd.read_excel('BiocharDS_V1.0.xlsx', sheet_name='Sheet1')
    explanation_data = pd.read_excel('BiocharDS_V1.0.xlsx', sheet_name='Explanation and unit')
    print("ファイル「BiocharDS_V1.0.xlsx」が正常に読み込まれました。")

except FileNotFoundError:
    # ファイルがない場合はアップロードを促す
    print("ファイル「BiocharDS_V1.0.xlsx」が見つかりません。アップロードしてください。")
    print("ファイル「BiocharDS_V1.0.xlsx」をアップロードしてください。Sheet1とExplanation and unitの2つのシートが必要です。")
    uploaded = files.upload()

    # アップロードされたファイルを確認
    if 'BiocharDS_V1.0.xlsx' in uploaded:
        biochar_data = pd.read_excel('BiocharDS_V1.0.xlsx', sheet_name='Sheet1')
        explanation_data = pd.read_excel('BiocharDS_V1.0.xlsx', sheet_name='Explanation and unit')
        print("ファイル「BiocharDS_V1.0.xlsx」が正常にアップロードされました。")
    else:
        raise FileNotFoundError("必要なファイル「BiocharDS_V1.0.xlsx」がアップロードされませんでした。")

# 列名のスペースを削除
biochar_data.columns = [col.strip() for col in biochar_data.columns]
explanation_data.columns = [col.strip() for col in explanation_data.columns]

# イネ（Rice）のデータのみを選択
rice_data = biochar_data[(biochar_data['CropType'] == 'Rice') & (biochar_data['Treatment'] == 'B')].copy()

# カウント情報を表示
print(f"元のデータサイズ: {biochar_data.shape}")
print(f"イネのデータサイズ: {rice_data.shape}")

# 目的変数の作成: CropYield_T - CropYield_CK
rice_data['Yield_Difference'] = rice_data['CropYield_T'] - rice_data['CropYield_CK']

# 目的変数作成後のデータサイズを確認
print(f"目的変数作成後のデータサイズ: {rice_data.shape}")

# 目的変数が欠損しているインスタンスの数を確認
missing_target_count = rice_data['Yield_Difference'].isna().sum()
print(f"目的変数が欠損しているインスタンス数: {missing_target_count}")

# 目的変数が欠損しているインスタンスを除去する前のデータ数を記録
before_dropna_count = len(rice_data)

# 目的変数が欠損しているインスタンスを除去
rice_data = rice_data.dropna(subset=['Yield_Difference'])

# 欠損値除去後のデータ数を記録
after_dropna_count = len(rice_data)
dropped_na_count = before_dropna_count - after_dropna_count

print(f"NaN除去前のデータ数: {before_dropna_count}")
print(f"NaN除去後のデータ数: {after_dropna_count}")
print(f"NaN除去により削減されたデータ数: {dropped_na_count}")

# 外れ値除去: 目的変数の値域に基づくフィルタリング
print("Starting fixed threshold filtering process...")

# フィルタリング前のデータ数を記録
before_filtering_count = len(rice_data)

# 目的変数の値が-3以下あるいは5以上のデータを除去
filtered_rice_data = rice_data[(rice_data['Yield_Difference'] > -3) & (rice_data['Yield_Difference'] < 8)]

# 除外されたデータ数を計算
excluded_count = before_filtering_count - len(filtered_rice_data)
excluded_pct = (excluded_count / before_filtering_count) * 100

# 結果を表示
print(f"フィルタリング前のデータ数: {before_filtering_count}")
print(f"フィルタリング後のデータ数: {len(filtered_rice_data)}")
print(f"除外されたデータ数: {excluded_count}件 ({excluded_pct:.2f}%)")

# 元の変数に代入
rice_data = filtered_rice_data

# 目的変数のフィルタリング後のデータ数を記録
after_filtering_count = len(rice_data)
filtered_count = before_filtering_count - after_filtering_count

print(f"目的変数のフィルタリング前のデータ数: {before_filtering_count}")
print(f"目的変数のフィルタリング後のデータ数: {after_filtering_count}")
print(f"目的変数のフィルタリングにより削減されたデータ数: {filtered_count}")

# 説明変数の選択
features = [
    'Temperature', 'Precipitation',#'WetnessIndex',
    'SOC_initial', 'TN_initial', 'TP_initial',
    'NH4_N_initial', 'NO3_N_initial', 'OlsenP_initial', 'pH_initial',
    'BD_initial', 'CEC_initial', 'Texture_initial', 'Type_biochar',
    'ParticleSize_biochar','BD_biochar', 'PyrolysisTemperature_biochar',
    'Ash_biochar','C_biochar', 'N_biochar', 'P_biochar', 'CNRatio_biochar',
    'pH_biochar', 'CEC_biochar',
    'BiocharAddition', 'TrialDuration'
]

# 説明変数のコピーを作成
all_features = features.copy()

# 最終的なデータサイズの報告
final_dataset_size = len(rice_data)
print(f"最終的なデータセットサイズ: {final_dataset_size}")

# データ処理の総括を表示
print("\n================ データ処理の総括 ================")
print(f"1. 元のイネデータ数: {before_dropna_count}")
print(f"2. 目的変数欠損による除外: {dropped_na_count}")
print(f"3. 目的変数のフィルタリングによる除外: {filtered_count}")
print(f"   - 除外基準: 目的変数が-3以下または5以上のデータ")
print(f"4. 最終データ数: {final_dataset_size}")
print(f"総削減数: {before_dropna_count - final_dataset_size}")
print("================================================\n")

# フィルタリング後の目的変数の分布を可視化
plt.figure(figsize=(8, 8))

plt.subplot(2, 1, 1)
plt.boxplot(rice_data['Yield_Difference'])
plt.title('Distribution of Yield Difference After Filtering')
plt.ylabel('Yield Difference (CropYield_T - CropYield_CK)')
# 分布の範囲を明示
plt.axhline(y=-3, color='r', linestyle='--', alpha=0.5)
plt.axhline(y=4, color='r', linestyle='--', alpha=0.5)

plt.subplot(2, 1, 2)
plt.hist(rice_data['Yield_Difference'], bins=20)
plt.title('Histogram of Yield Difference After Filtering')
plt.xlabel('Yield Difference')
plt.ylabel('Frequency')
# ヒストグラムにも範囲の境界線を追加
plt.axvline(x=-3, color='r', linestyle='--', alpha=0.5)
plt.axvline(x=4, color='r', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.savefig(os.path.join('section1_data_preparation', 'Yield_Difference_Distribution_After_Filtering.png'))
plt.close()

# カラム名の修正（Silt _initialをSilt_initialに変更）
if 'Silt _initial' in rice_data.columns:
    rice_data = rice_data.rename(columns={'Silt _initial': 'Silt_initial'})

# Texture_initialの数値への変換
if 'Texture_initial' in rice_data.columns:
    texture_mapping = {
        # 細粒土壌（粘土質）- 粒子が最も小さい
        'Clay': 1,             # 粘土
        'Silty clay': 1,       # シルト質粘土
        'Sandy clay': 1,       # 砂質粘土

        # 中粒土壌（壌土質）- 中間の粒子サイズ
        'Silty clay loam': 2,  # シルト質粘土質壌土
        'Clay loam': 2,        # 粘土質壌土
        'Sandy clay loam': 2,  # 砂質粘土質壌土
        'Silty loam': 2,       # シルト質壌土
        'Loam': 2,             # 壌土
        'Loamy clay': 2,       # 壌土質粘土

        # 粗粒土壌（砂質）- 粒子が最も大きい
        'Sandy loam': 3,       # 砂質壌土
        'Loamy sand': 3,       # 壌土質砂
        'Sand': 3              # 砂
    }

    # 前処理：未知の値をNaNとして扱う
    def map_texture(texture):
        if pd.isna(texture) or texture == '' or not isinstance(texture, str):
            return np.nan
        texture = texture.strip()
        return texture_mapping.get(texture, np.nan)

    rice_data['Texture_initial'] = rice_data['Texture_initial'].apply(map_texture)

# PyrolysisTemperature_biocharの数値への変換
if 'PyrolysisTemperature_biochar' in rice_data.columns:
    def fixed_convert_pyrolysis_temp(temp_val):
        # NaNの場合
        if pd.isna(temp_val):
            return np.nan

        # すでに数値型の場合はそのまま返す
        if isinstance(temp_val, (int, float)):
            return float(temp_val)

        # 文字列型の処理
        if isinstance(temp_val, str):
            temp_str = temp_val.strip()

            if temp_str == '':
                return np.nan

            # 範囲の処理
            if '-' in temp_str:
                try:
                    parts = temp_str.split('-')
                    low = float(parts[0].strip())
                    high = float(parts[1].strip())
                    return (low + high) / 2
                except:
                    return np.nan

            # 単一値の処理
            try:
                return float(temp_str)
            except:
                return np.nan

        return np.nan

    rice_data['PyrolysisTemperature_biochar'] = rice_data['PyrolysisTemperature_biochar'].apply(fixed_convert_pyrolysis_temp)

# Type_biocharの数値への変換
if 'Type_biochar' in rice_data.columns:
    # 論文を参考にした分類: W-R (Wood residues), ST-R (straw residues),
    # L-M (livestock manure), and SH-R (shell residues)
    biochar_mapping = {
        # 1: 木質系バイオマス (Wood residues)
        'Wood': 1,
        'Eucalyptus wood': 1,
        'Pine chip': 1,
        'Pine': 1,
        'Walnut shell': 1,
        'Wood sawdust': 1,
        'Spruce and Beech': 1,
        'Jarrah': 1,
        'Trunks and branches': 1,
        'Acacia tree': 1,
        'Poplar and plane tree wood': 1,
        'Subabul stem and twigs': 1,
        'Hardwood woodchips': 1,
        'Hardwood woodchips(primarily oak,elm and hickory)': 1,
        'Gliricidia sepium': 1,
        'Eupatorium plant': 1,
        'Eupatorium adenophorum': 1,
        'Acacia mangium bark': 1,
        'Elaeagnus angustifolia residue': 1,
        'Green Waste': 1,
        'Olive tree prunings': 1,
        'Apple branches': 1,

        # 2: わら系残渣 (Straw residues)
        'Wheat straw': 2,
        'Rice straw': 2,
        'Rice Straw': 2,
        'Maize straw': 2,
        'Rapeseed straw': 2,
        'Rape straw': 2,
        'peanut straw': 2,
        'Peanut straw': 2,
        'Soybean straw': 2,
        'Barley straw': 2,
        'Cotton straw': 2,
        'Cotton stock': 2,
        'Tobacco straw': 2,
        'Vicia faba straw': 2,
        'Faba bean straw': 2,
        'Urad straw': 2,
        'Mungbean straw': 2,
        'Pea straw': 2,
        'Straw': 2,
        'Wheat and rice straw': 2,
        'Bagasse': 2,
        'Reed': 2,
        ' Reed': 2,
        'Miscanthus': 2,
        'Vine': 2,
        'Ugarcane straw': 2,
        'Canola straw': 2,
        'Potato straw': 2,
        'Capsicum straw': 2,
        'Maize silage': 2,
        'Rice chaff': 2,
        'Bamboo': 2,
        'Bamboo ': 2,
        'Cassava stem': 2,
        'cyperus esculentus straw': 2,
        'Crop straw': 2,
        'Vegetable waste': 2,
        'Sorghum biomass': 2,

        # 3: 畜産廃棄物 (Livestock manure)
        'Cow manure': 3,
        'Cattle feedlot': 3,
        'Chicken manure': 3,
        'Poultry litter': 3,
        'Poultry manure': 3,
        'Pig manure': 3,
        'Diseased pig': 3,
        'Quail litter': 3,
        'Manure': 3,
        'Sludge': 3,
        'sludge': 3,
        'Sewage sludge': 3,
        'Green manure compost': 3,
        'Domestic waste': 3,
        'Municipal biowaste': 3,
        'Mushroom': 3,
        'Mixed': 3,
        'Charcoal': 3,

        # 4: 殻系残渣 (Shell residues)
        'Rice husk': 4,
        'Peanut hull': 4,
        'peanut hull': 4,
        'Coconut shell': 4,
        'Coconut husk': 4,
        'Palm kernel shell': 4,
        'Maize cob': 4,
        'Maize cob ': 4,
        'Maize cob-residues': 4,
        'Oat husk': 4,
        'Oat hull': 4,
        'wheat bran': 4,
        'Nut shell': 4,
        'Cottonseed husk': 4,
        'Cottonseed husk ': 4,
        'Cacao shell': 4,
        'Grape pomace': 4,
        'Mixture of rice husk and shell of cotton seed': 4,
        'mixture of rice husk and shell of cotton seed': 4
    }

    # 前処理：未知の値をNaNとして扱う
    def map_biochar_type(biochar_type):
        if pd.isna(biochar_type) or biochar_type == '' or not isinstance(biochar_type, str):
            return np.nan
        biochar_type = biochar_type.strip()
        return biochar_mapping.get(biochar_type, np.nan)

    # 元のカラムをコピーして保持
    rice_data['Type_biochar_original'] = rice_data['Type_biochar']

    # Type_biocharを数値に変換
    rice_data['Type_biochar'] = rice_data['Type_biochar'].apply(map_biochar_type)

    # 変換結果のサマリーを表示
    print("\n=== バイオ炭タイプの変換結果 ===")
    print(f"1. 木質系バイオマス (Wood residues): {(rice_data['Type_biochar'] == 1).sum()}件")
    print(f"2. わら系残渣 (Straw residues): {(rice_data['Type_biochar'] == 2).sum()}件")
    print(f"3. 畜産廃棄物 (Livestock manure): {(rice_data['Type_biochar'] == 3).sum()}件")
    print(f"4. 殻系残渣 (Shell residues): {(rice_data['Type_biochar'] == 4).sum()}件")
    print(f"未分類/欠損値: {rice_data['Type_biochar'].isna().sum()}件")

# 説明変数と目的変数の準備
X = rice_data[features]
y = rice_data['Yield_Difference']

# 各特徴量の欠損値の割合を計算
missing_percentage = X.isnull().mean() * 100
print("Missing data percentage for each feature (%):")
for feature, percentage in missing_percentage.items():
    print(f"{feature}: {percentage:.2f}%")

# 欠損値が50%以上の特徴量を除外
high_missing_features = missing_percentage[missing_percentage >= 50].index.tolist()
if high_missing_features:
    print(f"\nExcluding features with missing data ≥ 50%: {high_missing_features}")
    X = X.drop(columns=high_missing_features)

# 残りの説明変数の欠損値の数を確認
print(f"\nNumber of missing values in explanatory variables after processing: {X.isnull().sum().sum()}")

# インデックスを明示的に保存
original_index = X.index.copy()

# KNN補間 (k=5)
imputer = KNNImputer(n_neighbors=5)
X_imputed = imputer.fit_transform(X)

# データフレームに戻す - 重要: 元のインデックスを保持する
X_processed = pd.DataFrame(X_imputed, columns=X.columns, index=original_index)

# スケーリングを行わないように変更
# X_processed をそのまま使用する

# X_original と X_processed は同じインデックスを持っているので、loc の問題は解決されるはず
X_original = rice_data[features].copy()
# 欠損値が多い特徴量を除外した後の列名に合わせる
if high_missing_features:
    X_original = X_original.drop(columns=high_missing_features)
# X_original と X_processed は同じインデックスを持っているので、loc の問題は解決されるはず

# 確認のためのコード
print(f"X_original index shape: {X_original.index.shape}")
print(f"X_processed index shape: {X_processed.index.shape}")
print(f"共通するインデックスの数: {len(set(X_original.index) & set(X_processed.index))}")

# すべての説明変数および目的変数のヒストグラムを作成
print("\n補間後の説明変数と目的変数のヒストグラムを作成中...")

# 説明変数のヒストグラム
for feature in X_processed.columns:
    plt.figure(figsize=(8, 6))
    plt.hist(X_processed[feature], bins=20, color='skyblue', edgecolor='black')
    plt.title(f'Histogram of {feature} (After Imputation)')
    plt.xlabel(feature)
    plt.ylabel('Frequency')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f'{histograms_dir}/hist_{feature}.png')
    plt.close()

# 目的変数のヒストグラム
plt.figure(figsize=(8, 6))
plt.hist(y, bins=20, color='lightgreen', edgecolor='black')
plt.title('Histogram of Yield_Difference (Target Variable)')
plt.xlabel('Yield_Difference')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(f'{histograms_dir}/hist_Yield_Difference.png')
plt.close()

# 補間前の元データの説明変数のヒストグラム (参考用)
for feature in X.columns:
    plt.figure(figsize=(8, 6))
    plt.hist(X[feature].dropna(), bins=20, color='salmon', edgecolor='black')
    plt.title(f'Histogram of {feature} (Before Imputation)')
    plt.xlabel(feature)
    plt.ylabel('Frequency')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f'{histograms_dir}/hist_{feature}_before_imputation.png')
    plt.close()

print(f"ヒストグラムを {histograms_dir} フォルダに保存しました。")

# 国別のデータ
countries = rice_data['Country'].unique()
print(f"データに含まれる国のリスト: {countries}")

country_instances = {}
for country in countries:
    country_instances[country] = rice_data[rice_data['Country'] == country].index
    print(f"{country}のデータ数: {len(country_instances[country])}")

Created folder: section1_data_preparation
Created folder: section2_model_comparison
Created folder: section3_final_model
Created folder: section4_predictions_analysis
Created folder: section5_shap_analysis
Created folder: section6_interaction_analysis
Created folder: section7_feature_pdp
Created folder: section1_data_preparation/histograms
Created folder: section4_predictions_analysis/extreme_values
ファイル「BiocharDS_V1.0.xlsx」が見つかりません。アップロードしてください。
ファイル「BiocharDS_V1.0.xlsx」をアップロードしてください。Sheet1とExplanation and unitの2つのシートが必要です。


Saving BiocharDS_V1.0.xlsx to BiocharDS_V1.0.xlsx
ファイル「BiocharDS_V1.0.xlsx」が正常にアップロードされました。
元のデータサイズ: (2438, 209)
イネのデータサイズ: (327, 209)
目的変数作成後のデータサイズ: (327, 210)
目的変数が欠損しているインスタンス数: 40
NaN除去前のデータ数: 327
NaN除去後のデータ数: 287
NaN除去により削減されたデータ数: 40
Starting fixed threshold filtering process...
フィルタリング前のデータ数: 287
フィルタリング後のデータ数: 285
除外されたデータ数: 2件 (0.70%)
目的変数のフィルタリング前のデータ数: 287
目的変数のフィルタリング後のデータ数: 285
目的変数のフィルタリングにより削減されたデータ数: 2
最終的なデータセットサイズ: 285

================ データ処理の総括 ================
1. 元のイネデータ数: 327
2. 目的変数欠損による除外: 40
3. 目的変数のフィルタリングによる除外: 2
   - 除外基準: 目的変数が-3以下または5以上のデータ
4. 最終データ数: 285
総削減数: 42


=== バイオ炭タイプの変換結果 ===
1. 木質系バイオマス (Wood residues): 8件
2. わら系残渣 (Straw residues): 230件
3. 畜産廃棄物 (Livestock manure): 10件
4. 殻系残渣 (Shell residues): 36件
未分類/欠損値: 1件
Missing data percentage for each feature (%):
Temperature: 15.09%
Precipitation: 8.77%
SOC_initial: 14.74%
TN_initial: 10.18%
TP_initial: 72.28%
NH4_N_initial: 91.58%
NO3_N_initial: 96.49%
OlsenP_initial: 64.56%
pH_initial: 10.18%
BD_in

  フォールド 4/5 処理中...
  フォールド 5/5 処理中...
    RMSE: 0.9838, R²: 0.1701
  パラメータ評価中: layers=3, units=32, dropout=0.2, lr=0.001, batch_size=16
  フォールド 1/5 処理中...
  フォールド 2/5 処理中...
  フォールド 3/5 処理中...
  フォールド 4/5 処理中...
  フォールド 5/5 処理中...
    RMSE: 1.0944, R²: -0.0270
  パラメータ評価中: layers=3, units=32, dropout=0.2, lr=0.001, batch_size=32
  フォールド 1/5 処理中...
  フォールド 2/5 処理中...
  フォールド 3/5 処理中...
  フォールド 4/5 処理中...
  フォールド 5/5 処理中...
    RMSE: 1.0369, R²: 0.0779
  パラメータ評価中: layers=3, units=32, dropout=0.2, lr=0.01, batch_size=8
  フォールド 1/5 処理中...
  フォールド 2/5 処理中...
  フォールド 3/5 処理中...
  フォールド 4/5 処理中...
  フォールド 5/5 処理中...
    RMSE: 0.9722, R²: 0.1895
  パラメータ評価中: layers=3, units=32, dropout=0.2, lr=0.01, batch_size=16
  フォールド 1/5 処理中...
  フォールド 2/5 処理中...
  フォールド 3/5 処理中...
  フォールド 4/5 処理中...
  フォールド 5/5 処理中...
    RMSE: 0.9812, R²: 0.1744
  パラメータ評価中: layers=3, units=32, dropout=0.2, lr=0.01, batch_size=32
  フォールド 1/5 処理中...
  フォールド 2/5 処理中...
  フォールド 3/5 処理中...
  フォールド 4/5 処理中...
  フォールド 5/5 処理中...


#機械学習モデルの構築

機械学習モデルの精度の比較

In [ ]:
# ============= ２．機械学習モデルの精度の比較 =============

# 必要なインポートを追加
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.cross_decomposition import PLSRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
from itertools import product

# 警告を無視
warnings.filterwarnings('ignore')

# モデルの評価関数
def evaluate_model(y_true, y_pred):
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    rrmse = rmse / np.mean(np.abs(y_true)) * 100 if np.mean(np.abs(y_true)) != 0 else np.nan
    return r2, rmse, rrmse

# 交差検証の実行関数
def run_cross_validation(model, X, y, kf):
    """
    与えられたモデルで交差検証を実行し、予測値を返す

    Parameters:
    -----------
    model : モデルインスタンス
    X : 特徴量データ
    y : 目的変数
    kf : KFold オブジェクト

    Returns:
    --------
    y_pred_cv : 交差検証の予測値
    """
    y_pred_cv = np.zeros_like(y)

    for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # モデルを訓練
        model.fit(X_train, y_train)

        # テストデータで予測
        y_pred_cv[test_idx] = model.predict(X_test)

    return y_pred_cv

# ニューラルネットワークのクロスバリデーション
def run_nn_cross_validation(X, y, kf, params):
    """
    ニューラルネットワークの交差検証を実行し、予測値を返す

    Parameters:
    -----------
    X : 特徴量データ
    y : 目的変数
    kf : KFold オブジェクト
    params : モデルパラメータ辞書

    Returns:
    --------
    y_pred_cv : 交差検証の予測値
    """
    y_pred_cv = np.zeros_like(y)

    for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
        print(f"  フォールド {fold+1}/5 処理中...")
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # 検証データをさらに分割
        X_train_split, X_val, y_train_split, y_val = train_test_split(
            X_train, y_train, test_size=0.2, random_state=42
        )

        # モデルを定義 (固定の3層構造)
        model = Sequential()

        # 入力層
        model.add(Dense(params['units'], input_shape=(X.shape[1],)))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(params['dropout']))

        # 隠れ層1
        model.add(Dense(params['units']))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        #model.add(Dropout(params['dropout']))

        # 隠れ層2
        model.add(Dense(params['units']))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        #model.add(Dropout(params['dropout']))

        # 出力層
        model.add(Dense(1))

        # コンパイル
        optimizer = keras.optimizers.Adam(learning_rate=params['learning_rate'])
        model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])

        # Early Stoppingを設定
        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
            verbose=0
        )

        # モデルを訓練
        model.fit(
            X_train_split, y_train_split,
            epochs=100,
            batch_size=params['batch_size'],
            validation_data=(X_val, y_val),
            callbacks=[early_stopping],
            verbose=0
        )

        # テストデータで予測
        y_pred_cv[test_idx] = model.predict(X_test, verbose=0).flatten()

        # メモリ解放
        tf.keras.backend.clear_session()

    return y_pred_cv

# ======================= 各アルゴリズムのハイパーパラメータ定義 =======================

# モデルとハイパーパラメータグリッドの定義
model_params = {
    'PLSR': {
        'model': PLSRegression,
        'params': {
            'n_components': [1, 3, 5, 10, 15]
        }
    },
    'Decision Tree': {
        'model': DecisionTreeRegressor,
        'params': {
            'max_depth': [5, 10, 15, None],
            'min_samples_split': [2, 5, 10]
        }
    },
    'Random Forest': {
        'model': RandomForestRegressor,
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [10, 20, None]
        }
    },
    'XGBoost': {
        'model': XGBRegressor,
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.1, 0.2]
        }
    },
    'Neural Network': {
        'params': {
            'layers': [3],        # 3層で固定
            'units': [32],        # 32ユニットで固定
            'dropout': [0.2],     # ドロップアウト率0.2で固定
            'learning_rate': [0.001, 0.01, 0.1],  # 学習率のみ探索
            'batch_size': [8, 16, 32]              # バッチサイズのみ探索
        }
    }
}

# 結果を格納するためのデータフレーム
results = pd.DataFrame(columns=['Algorithm', 'Best_Params', 'CV_R2', 'CV_RMSE', 'CV_RRMSE'])

# 5分割交差検証
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# データをnumpy配列に変換
X_data = X_processed.values if isinstance(X_processed, pd.DataFrame) else X_processed
y_data = y.values if isinstance(y, pd.Series) else y

print("\n======= 5分割交差検証によるモデル評価（シンプル版） =======")

# 各アルゴリズムの評価
for name, config in model_params.items():
    print(f"\n{name}の評価中...")

    # ニューラルネットワークの場合
    if name == 'Neural Network':
        best_rmse = float('inf')
        best_params = None
        best_r2 = -float('inf')
        best_rrmse = float('inf')
        best_y_pred = None

        # ハイパーパラメータの組み合わせごとに評価
        param_list = list(product(
            config['params']['layers'],
            config['params']['units'],
            config['params']['dropout'],
            config['params']['learning_rate'],
            config['params']['batch_size']
        ))

        for layers, units, dropout, lr, batch_size in param_list:
            print(f"  パラメータ評価中: layers={layers}, units={units}, dropout={dropout}, lr={lr}, batch_size={batch_size}")

            params = {
                'layers': layers,
                'units': units,
                'dropout': dropout,
                'learning_rate': lr,
                'batch_size': batch_size
            }

            # 交差検証を実行
            y_pred_cv = run_nn_cross_validation(X_data, y_data, kf, params)

            # 評価指標を計算
            r2, rmse, rrmse = evaluate_model(y_data, y_pred_cv)

            print(f"    RMSE: {rmse:.4f}, R²: {r2:.4f}")

            # 最良のモデルを更新
            if rmse < best_rmse:
                best_rmse = rmse
                best_r2 = r2
                best_rrmse = rrmse
                best_params = params
                best_y_pred = y_pred_cv

        # 最良のパラメータと結果を表示
        print(f"\n最良のパラメータ: {best_params}")
        print(f"CV R²: {best_r2:.4f}, CV RMSE: {best_rmse:.4f}, CV RRMSE: {best_rrmse:.2f}%")

    # その他のアルゴリズムの場合
    else:
        best_rmse = float('inf')
        best_params = None
        best_r2 = -float('inf')
        best_rrmse = float('inf')
        best_y_pred = None

        # パラメータの組み合わせを生成
        param_keys = list(config['params'].keys())
        param_values = list(config['params'].values())
        param_combinations = list(product(*param_values))

        # 各パラメータの組み合わせを評価
        for params_tuple in param_combinations:
            params_dict = {k: v for k, v in zip(param_keys, params_tuple)}
            param_str = ", ".join([f"{k}={v}" for k, v in params_dict.items()])
            print(f"  パラメータ評価中: {param_str}")

            # モデルを初期化
            if name == 'PLSR':
                # PLSRは初期化方法が異なるので特別扱い
                model = config['model'](**params_dict)
            else:
                model = config['model'](random_state=42, **params_dict)

            # 交差検証を実行
            y_pred_cv = run_cross_validation(model, X_data, y_data, kf)

            # 評価指標を計算
            r2, rmse, rrmse = evaluate_model(y_data, y_pred_cv)

            print(f"    RMSE: {rmse:.4f}, R²: {r2:.4f}")

            # 最良のモデルを更新
            if rmse < best_rmse:
                best_rmse = rmse
                best_r2 = r2
                best_rrmse = rrmse
                best_params = params_dict
                best_y_pred = y_pred_cv

        # 最良のパラメータと結果を表示
        print(f"\n最良のパラメータ: {best_params}")
        print(f"CV R²: {best_r2:.4f}, CV RMSE: {best_rmse:.4f}, CV RRMSE: {best_rrmse:.2f}%")

    # 交差検証による予測と実測値の散布図
    plt.figure(figsize=(8, 8))
    plt.scatter(y_data, best_y_pred, alpha=0.6)

    min_val = min(min(y_data), min(best_y_pred))
    max_val = max(max(y_data), max(best_y_pred))

    # 1:1ラインを追加
    plt.plot([min_val, max_val], [min_val, max_val], 'r--')

    plt.xlabel('Observed')
    plt.ylabel('Cross-Validation Predicted')
    plt.title(f'{name}: Cross-Validation Predictions vs Observed\nR² = {best_r2:.4f}, RMSE = {best_rmse:.4f}')
    plt.grid(True, alpha=0.3)
    plt.savefig(os.path.join('section2_model_comparison', f'{name}_cv_scatter_simple.png'))
    plt.close()

    # 結果をデータフレームに追加
    results = pd.concat([results, pd.DataFrame({
        'Algorithm': [name],
        'Best_Params': [str(best_params)],
        'CV_R2': [best_r2],
        'CV_RMSE': [best_rmse],
        'CV_RRMSE': [best_rrmse]
    })], ignore_index=True)

# 結果を保存
results = results.sort_values('CV_R2', ascending=False)
results.to_excel(os.path.join('section2_model_comparison', 'cross_validation_results_simple.xlsx'), index=False)
print("\n交差検証による比較結果:")
print(results)

# 決定係数の棒グラフ作成
plt.figure(figsize=(10, 6))
ax = sns.barplot(x='Algorithm', y='CV_R2', data=results, palette='viridis')
plt.title('5-fold cross validation')
plt.ylabel('R²')
plt.xlabel('Algorithm')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# 各バーの上に値を表示
for i, v in enumerate(results['CV_R2']):
    ax.text(i, v + 0.01, f'{v:.4f}', ha='center')

plt.tight_layout()
plt.savefig(os.path.join('section2_model_comparison', 'algorithm_comparison_cv_r2_simple.png'))
plt.close()

# RMSEの棒グラフ作成
plt.figure(figsize=(10, 6))
ax = sns.barplot(x='Algorithm', y='CV_RMSE', data=results, palette='coolwarm_r')
plt.title('5-fold cross validation')
plt.ylabel('RMSE')
plt.xlabel('Algorithm')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# 各バーの上に値を表示
for i, v in enumerate(results['CV_RMSE']):
    ax.text(i, v + 0.01, f'{v:.4f}', ha='center')

plt.tight_layout()
plt.savefig(os.path.join('section2_model_comparison', 'algorithm_comparison_cv_rmse_simple.png'))
plt.close()

# RRMSEの棒グラフ作成
plt.figure(figsize=(10, 6))
ax = sns.barplot(x='Algorithm', y='CV_RRMSE', data=results, palette='coolwarm_r')
plt.title('5-fold cross validation')
plt.ylabel('RRMSE (%)')
plt.xlabel('Algorithm')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# 各バーの上に値を表示
for i, v in enumerate(results['CV_RRMSE']):
    ax.text(i, v + 0.5, f'{v:.2f}%', ha='center')

plt.tight_layout()
plt.savefig(os.path.join('section2_model_comparison', 'algorithm_comparison_cv_rrmse_simple.png'))
plt.close()

SHAP関連解析用の機械学習モデルを全データにより構築

In [ ]:
# ============= ３．最終モデルの構築 =============

# 2で保存した最適なランダムフォレストのパラメータを使用
print("\nランダムフォレストの最終モデル構築中...")

# 交差検証結果から最適なパラメータを読み込む
try:
    cv_results = pd.read_excel(os.path.join('section2_model_comparison', 'cross_validation_results_simple.xlsx'))

    # Random Forestモデルの結果を取得
    rf_results = cv_results[cv_results['Algorithm'] == 'Random Forest']

    if not rf_results.empty:
        # ベストパラメータの文字列を取得してdictに変換
        best_params_str = rf_results.iloc[0]['Best_Params']

        # 文字列形式のパラメータを辞書に変換
        # 例: "{'n_estimators': 100, 'max_depth': 20}" → {'n_estimators': 100, 'max_depth': 20}
        import ast
        best_rf_params = ast.literal_eval(best_params_str)

        print(f"セクション2から取得した最適パラメータ: {best_rf_params}")
    else:
        # Random Forestの結果が見つからない場合はデフォルトパラメータを使用
        print("警告: Random Forestの結果が見つかりません。デフォルトパラメータを使用します。")
        best_rf_params = {'n_estimators': 100, 'max_depth': None}
except Exception as e:
    print(f"警告: 交差検証結果の読み込みでエラーが発生しました: {e}")
    print("デフォルトパラメータを使用します。")
    best_rf_params = {'n_estimators': 100, 'max_depth': None}

# 最終モデルの構築
final_rf_model = RandomForestRegressor(random_state=42, **best_rf_params)
final_rf_model.fit(X_processed, y)

# 訓練データでの予測
y_train_pred = final_rf_model.predict(X_processed)
r2_train, rmse_train, rrmse_train = evaluate_model(y, y_train_pred)

print(f"訓練データでの決定係数 (R²): {r2_train:.4f}")
print(f"訓練データでのRMSE: {rmse_train:.4f}")
print(f"訓練データでのRRMSE: {rrmse_train:.4f}%")

# 訓練データの散布図
plt.figure(figsize=(8, 8))
plt.scatter(y, y_train_pred)
min_val = min(min(y), min(y_train_pred))
max_val = max(max(y), max(y_train_pred))

# 正方形のプロットエリアを確保するために軸の範囲を調整
overall_min = min(min_val, min_val)
overall_max = max(max_val, max_val)
plt.xlim(overall_min, overall_max)
plt.ylim(overall_min, overall_max)

plt.plot([overall_min, overall_max], [overall_min, overall_max], 'r--')
plt.xlabel('Observed')
plt.ylabel('Predicted')
plt.title(f'Final Random Forest Model: Predicted vs Observed (R² = {r2_train:.4f}, RMSE = {rmse_train:.4f}, RRMSE = {rrmse_train:.2f}%)')
plt.grid(True)
plt.savefig(os.path.join('section3_final_model', 'RandomForest_Final_scatter.png'))
plt.close()

#SHAP関連解析

予測値の上位3サンプルと下位3サンプル（国の重複を避ける）のSHAP値の可視化

In [ ]:
# ============= ４．すべてのインスタンスの収量差予測の表示と極値の分析 =============

# SHAP値による解釈
print("\nSHAP値による解釈を開始...")

# SHAP値の計算
explainer = shap.TreeExplainer(final_rf_model)
shap_values = explainer.shap_values(X_processed)

# ======= 4.1 すべてのインスタンスの予測値を世界地図上にプロット =======

print("\n1. すべてのインスタンスの予測値を世界地図上にプロット中...")

# 必要なライブラリのインポート
try:
    import matplotlib.pyplot as plt
    import cartopy.crs as ccrs
    import cartopy.feature as cfeature
    print("必要なライブラリは既にインストールされています。")
except ImportError:
    print("必要なライブラリをインストールしています...")
    !pip install matplotlib cartopy
    import matplotlib.pyplot as plt
    import cartopy.crs as ccrs
    import cartopy.feature as cfeature

# 各インスタンスの位置情報と予測値を抽出
all_instances_data = []
for idx in X_processed.index:
    if idx in rice_data.index:
        instance = rice_data.loc[idx]
        country = instance['Country'] if 'Country' in instance else 'Unknown'

        # 緯度と経度の情報があるか確認
        if 'Latitude' in instance and 'Longitude' in instance:
            lat = instance['Latitude']
            lon = instance['Longitude']

            if pd.notna(lat) and pd.notna(lon):
                # 予測値と実測値を取得
                pred_value = final_rf_model.predict(X_processed.loc[idx:idx])[0]
                actual_value = y.loc[idx]

                all_instances_data.append({
                    'Index': idx,
                    'Country': country,
                    'Latitude': lat,
                    'Longitude': lon,
                    'PredictedValue': pred_value,
                    'ActualValue': actual_value,
                    'AbsPredicted': abs(pred_value)
                })

# データが見つからない場合はデフォルトの国の位置情報を使用
if len(all_instances_data) < len(X_processed) * 0.5:  # データの半分以上が位置情報を持っていない場合
    print("\n多くのインスタンスに位置情報がありません。国のデフォルト位置を使用します...")

    # 国のデフォルト位置
    country_locations = {
        'China': {'lat': 35.86, 'lon': 104.19},
        'India': {'lat': 20.59, 'lon': 78.96},
        'Japan': {'lat': 36.20, 'lon': 138.25},
        'Indonesia': {'lat': -0.79, 'lon': 113.92},
        'Brazil': {'lat': -14.24, 'lon': -51.93},
        'United States': {'lat': 37.09, 'lon': -95.71},
        'Australia': {'lat': -25.27, 'lon': 133.77},
        'Italy': {'lat': 41.87, 'lon': 12.57},
        'Spain': {'lat': 40.46, 'lon': -3.75},
        'France': {'lat': 46.23, 'lon': 2.21},
        'Germany': {'lat': 51.17, 'lon': 10.45},
        'United Kingdom': {'lat': 55.38, 'lon': -3.44},
        'Canada': {'lat': 56.13, 'lon': -106.35},
        'Russia': {'lat': 61.52, 'lon': 105.32},
        'South Korea': {'lat': 35.91, 'lon': 127.77},
        'Philippines': {'lat': 12.88, 'lon': 121.77},
        'Vietnam': {'lat': 14.06, 'lon': 108.28},
        'Thailand': {'lat': 15.87, 'lon': 100.99},
        'Malaysia': {'lat': 4.21, 'lon': 101.98},
        'Bangladesh': {'lat': 23.68, 'lon': 90.35},
        'Myanmar': {'lat': 21.92, 'lon': 95.96},
        'Pakistan': {'lat': 30.38, 'lon': 69.35},
        'Sri Lanka': {'lat': 7.87, 'lon': 80.77},
        'Nepal': {'lat': 28.39, 'lon': 84.12},
        'Nigeria': {'lat': 9.08, 'lon': 8.67},
        'Egypt': {'lat': 26.82, 'lon': 30.80},
        'Turkey': {'lat': 38.96, 'lon': 35.24},
        'Iran': {'lat': 32.43, 'lon': 53.69},
        'Mexico': {'lat': 23.63, 'lon': -102.55},
        'Argentina': {'lat': -38.42, 'lon': -63.62},
        'Colombia': {'lat': 4.57, 'lon': -74.30},
        'Peru': {'lat': -9.19, 'lon': -75.00},
        'Chile': {'lat': -35.68, 'lon': -71.54}
    }

    # 国ごとにインスタンスをグループ化
    country_instances = {}
    for idx in X_processed.index:
        country = rice_data.loc[idx, 'Country'] if 'Country' in rice_data.columns and idx in rice_data.index else 'Unknown'

        if country not in country_instances:
            country_instances[country] = []
        country_instances[country].append(idx)

    # 各国のインスタンスに対して位置情報を追加
    all_instances_data = []
    for country, instances in country_instances.items():
        if country in country_locations:
            lat = country_locations[country]['lat']
            lon = country_locations[country]['lon']

            # 同じ国の複数のインスタンスを少しオフセットして表示
            import random
            random.seed(42)  # 再現性のため

            for idx in instances:
                if idx in X_processed.index and idx in y.index:
                    # 小さなランダムオフセットを適用
                    offset_scale = min(3, max(1, len(instances) / 10))
                    lat_offset = (random.random() - 0.5) * offset_scale
                    lon_offset = (random.random() - 0.5) * offset_scale

                    # 予測値と実測値を取得
                    pred_value = final_rf_model.predict(X_processed.loc[idx:idx])[0]
                    actual_value = y.loc[idx] if idx in y.index else np.nan

                    all_instances_data.append({
                        'Index': idx,
                        'Country': country,
                        'Latitude': lat + lat_offset,
                        'Longitude': lon + lon_offset,
                        'PredictedValue': pred_value,
                        'ActualValue': actual_value,
                        'AbsPredicted': abs(pred_value)
                    })
        else:
            print(f"警告: {country}のデフォルト位置情報がありません。")

# データフレームに変換
if all_instances_data:
    all_df = pd.DataFrame(all_instances_data)
    print(f"合計 {len(all_df)} インスタンスの位置情報と予測値を取得しました。")

    # 世界地図の作成
    plt.figure(figsize=(14, 10))
    ax = plt.axes(projection=ccrs.PlateCarree())

    # 地図の基本要素を追加
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN, facecolor='lightblue')
    ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
    ax.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
    ax.set_global()

    # 各インスタンスをプロット
    marker_size = 10  # 統一サイズ

    # カラーマップの作成
    import matplotlib as mpl
    import numpy as np

    # -0.5から+2.5の範囲で青から赤のグラデーションを定義
    cmap = mpl.colors.LinearSegmentedColormap.from_list(
        'custom_diverging',
        [(0, 'darkblue'), (0.5, 'white'), (1, 'darkred')],
        N=256
    )

    # 値の範囲を-0.5から2.5に設定
    norm = mpl.colors.Normalize(vmin=-0.5, vmax=2.5)

    # 各インスタンスをプロット（色分けを修正）
    for _, row in all_df.iterrows():
        # 予測値に基づいて色を決定
        # normとcmapを使って、-0.5から2.5の範囲で青から赤への色を決定
        color = cmap(norm(row['PredictedValue']))

        # マーカープロット - 透明度は一定に
        ax.scatter(row['Longitude'], row['Latitude'], s=marker_size, color=color,
                  alpha=0.7, transform=ccrs.PlateCarree(), edgecolor='black', linewidth=0.3)

    # カラーバーの設定
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax, shrink=0.6, pad=0.02)
    cbar.set_ticklabels(['-0.5', '0', '1', '2', '2.5'])

    plt.title('All Rice Instances: Predicted Yield Difference\n(Blue = Negative, Red = Positive)', fontsize=14)
    plt.tight_layout()
    plt.savefig(os.path.join('section4_predictions_analysis', 'All_Instances_Prediction_Map.png'), dpi=300, bbox_inches='tight')
    plt.close()

    print("すべてのインスタンスの予測マップを 'section4_predictions_analysis/All_Instances_Prediction_Map.png' に保存しました。")

    # インスタンスデータをCSVに保存
    all_df.to_csv(os.path.join('section4_predictions_analysis', 'All_Instances_Predictions.csv'), index=False)
    print("すべてのインスタンスの予測データを 'section4_predictions_analysis/All_Instances_Predictions.csv' に保存しました。")
else:
    print("位置情報を持つインスタンスが見つかりませんでした。マップは生成されません。")

# ======= 4.2 極値（上位3件、下位3件）の選択 =======

print("\n2. 予測収量差の極値を選択中（上位3件、下位3件）...")

# データフレームが存在することを確認
if 'all_df' in locals() and not all_df.empty:
    # 予測値で降順と昇順にソート
    sorted_high = all_df.sort_values('PredictedValue', ascending=False)
    sorted_low = all_df.sort_values('PredictedValue', ascending=True)

    # 上位グループ内で国の重複を避けながら上位3件を取得
    top_3_countries = []
    top_3_instances = []

    for _, row in sorted_high.iterrows():
        if row['Country'] not in top_3_countries:
            top_3_countries.append(row['Country'])
            top_3_instances.append(row)
            if len(top_3_instances) >= 3:  # 3件に変更
                break

    # 下位グループ内で国の重複を避けながら下位3件を取得
    bottom_3_countries = []
    bottom_3_instances = []

    for _, row in sorted_low.iterrows():
        if row['Country'] not in bottom_3_countries:
            bottom_3_countries.append(row['Country'])
            bottom_3_instances.append(row)
            if len(bottom_3_instances) >= 3:  # 3件に変更
                break

    # 6個のインスタンスを一つのデータフレームに結合
    extremes_df = pd.DataFrame(top_3_instances + bottom_3_instances)

    # 確認のため、選択したインスタンスを表示
    print("\n選択された6個の極値インスタンス:")
    print("上位3件（高い収量差）:")
    for i, row in enumerate(top_3_instances, 1):
        print(f"{i}. {row['Country']}: 予測値 = {row['PredictedValue']:.4f}, 実測値 = {row['ActualValue']:.4f}")

    print("\n下位3件（低い収量差）:")
    for i, row in enumerate(bottom_3_instances, 1):
        print(f"{i}. {row['Country']}: 予測値 = {row['PredictedValue']:.4f}, 実測値 = {row['ActualValue']:.4f}")

# 極値インスタンスの地図を生成する部分を修正
# 中国の複数インスタンスのラベルが重ならないように調整

if 'extremes_df' in locals() and not extremes_df.empty:
    # 重なりを解消するための調整
    # まず極値データフレームのコピーを作成
    extremes_df_adjusted = extremes_df.copy()

    # 中国のインスタンスを特定
    china_instances = extremes_df_adjusted[extremes_df_adjusted['Country'] == 'China']

    # 中国のインスタンスが複数ある場合
    if len(china_instances) > 1:
        print(f"中国の複数インスタンス ({len(china_instances)}件) のラベル位置を調整します。")

        # 中国のインスタンスそれぞれに異なるオフセットを適用
        for i, idx in enumerate(china_instances.index):
            # 1つ目は左下、2つ目は右下など、異なる方向にオフセット
            if i == 0:
                # 1つ目のインスタンス: 左上方向にオフセット
                extremes_df_adjusted.loc[idx, 'Longitude'] -= 11.0
                extremes_df_adjusted.loc[idx, 'Latitude'] += 5.0
            elif i == 1:
                # 2つ目のインスタンス: 左下方向にオフセット
                extremes_df_adjusted.loc[idx, 'Longitude'] -= 11.0
                extremes_df_adjusted.loc[idx, 'Latitude'] -= 5.0
            else:
                # 3つ目以降（もしあれば）: 下方向に順次オフセット
                extremes_df_adjusted.loc[idx, 'Longitude'] += (i - 1) * 10.0
                extremes_df_adjusted.loc[idx, 'Latitude'] -= 5.0

    # 韓国のインスタンスを特定
    korea_instances = extremes_df_adjusted[extremes_df_adjusted['Country'] == 'Korea']

    # 韓国のインスタンスがある場合
    if len(korea_instances) > 0:
        print(f"韓国のインスタンス ({len(korea_instances)}件) のラベル位置を調整します。")

        # 韓国のインスタンスのラベルを右下方向にオフセット
        for idx in korea_instances.index:
            extremes_df_adjusted.loc[idx, 'Longitude'] += 10.0
            extremes_df_adjusted.loc[idx, 'Latitude'] -= 6.0

    # 世界地図の作成
    plt.figure(figsize=(14, 10))
    ax = plt.axes(projection=ccrs.PlateCarree())

    # 地図の基本要素を追加
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN, facecolor='lightblue')
    ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
    ax.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
    ax.set_global()

    # カラーマップの作成
    cmap = mpl.colors.LinearSegmentedColormap.from_list(
        'custom_diverging',
        [(0, 'darkblue'), (0.5, 'white'), (1, 'darkred')],
        N=256
    )

    # 値の範囲を-0.5から2.5に設定
    norm = mpl.colors.Normalize(vmin=-0.5, vmax=2.5)

    # 極値インスタンスをプロット
    marker_size = 20  # 統一サイズ

    for _, row in extremes_df.iterrows():
        # 元の位置にマーカーをプロット
        # 予測値に基づいて色を決定
        color = cmap(norm(row['PredictedValue']))

        # マーカープロット
        ax.scatter(row['Longitude'], row['Latitude'], s=marker_size, color=color, alpha=0.8,
                  transform=ccrs.PlateCarree(), edgecolor='black', linewidth=0.5)

    # 調整した位置にラベルを表示
    for i, row in extremes_df_adjusted.iterrows():
        # ラベルテキストの設定
        label_text = f"{row['Country']}\n({row['PredictedValue']:.2f})"

        # ラベルを追加
        ax.text(row['Longitude'], row['Latitude']+2, label_text,
                fontsize=9, ha='center', va='bottom', transform=ccrs.PlateCarree(),
                bbox=dict(facecolor='white', alpha=0.7, boxstyle='round,pad=0.2'))

    # カラーバー
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])

    cbar = plt.colorbar(sm, ax=ax, shrink=0.6, pad=0.02)
    cbar.set_label('Predicted Yield Difference')
    # -0.5, 0, 1, 2, 2.5のティック位置にラベルを設定
    cbar.set_ticks([-0.5, 0, 1, 2, 2.5])
    cbar.set_ticklabels(['-0.5', '0', '1', '2', '2.5'])

    plt.title('Top 3 Highest and Top 3 Lowest Predicted Yield Differences\n(By Country, No Duplicates Within Groups)', fontsize=14)
    plt.tight_layout()
    plt.savefig(os.path.join('section4_predictions_analysis', 'extreme_values', 'Extreme_Values_Map.png'), dpi=300, bbox_inches='tight')
    plt.close()

    print("\n調整した極値インスタンスの地図を 'section4_predictions_analysis/extreme_values/Extreme_Values_Map.png' に保存しました。")

    # ======= 4.3 選択した6個のインスタンスのWaterfall Plotを作成 =======

    print("\n3. 選択した6個の極値インスタンスのWaterfall Plotを作成中...")

    # 関数を定義：Waterfall Plotの作成
    def create_extreme_waterfall_plot(idx, country, rank_info, X_processed, X_original, shap_values, explainer):
        """
        極値インスタンスのWaterfall Plotを作成する関数
        """
        if idx in X_processed.index:
            try:
                # データポイントの取得
                instance_processed = X_processed.loc[idx:idx]
                instance_original = X_original.loc[idx:idx] if idx in X_original.index else None

                # インデックスの位置を取得
                idx_pos = X_processed.index.get_loc(idx)

                # SHAPウォーターフォールプロット
                plt.figure(figsize=(12, 10))
                waterfall = shap.waterfall_plot(
                    shap.Explanation(
                        values=shap_values[idx_pos],
                        base_values=explainer.expected_value,
                        data=instance_processed.iloc[0],
                        feature_names=X_processed.columns
                    ),
                    show=False
                )

                # プロットのタイトル情報
                pred_value = final_rf_model.predict(instance_processed)[0]
                actual_value = y.loc[idx] if idx in y.index else np.nan

                title = f'SHAP Waterfall Plot: {country} ({rank_info})\n'
                title += f'Predicted: {pred_value:.2f}, Actual: {actual_value:.2f}\n'

                # 補間前の特徴量値があれば追加
                if instance_original is not None:
                    # インスタンスの主要な特徴を取得
                    feature_values = instance_original.iloc[0].to_dict()

                    # 重要な特徴量に絞って表示（すべてだと多すぎる）
                    # SHAP値の絶対値でソートして上位10個の特徴量のみ表示
                    shap_abs = np.abs(shap_values[idx_pos])
                    top_features_idx = np.argsort(shap_abs)[-10:][::-1]  # 上位10個
                    top_features = [X_processed.columns[i] for i in top_features_idx]

                    title += 'Top 10 feature values (before imputation):\n'
                    for i, feature in enumerate(top_features):
                        if i > 0 and i % 2 == 0:  # 2つごとに改行
                            title += '\n'
                        # NaN値の適切な表示
                        value = feature_values.get(feature, np.nan)
                        if pd.isna(value):
                            title += f'{feature}: NaN, '
                        else:
                            title += f'{feature}: {value:.2f}, '

                plt.title(title, fontsize=10)
                plt.tight_layout()
                plt.savefig(os.path.join('section4_predictions_analysis', 'extreme_values', f'SHAP_Waterfall_{country}_{rank_info.replace(" ", "_")}.png'), bbox_inches='tight')
                plt.close()

                print(f"{country} ({rank_info}) のWaterfall Plotを作成しました。")
                return True
            except Exception as e:
                print(f"{country} ({rank_info}) のWaterfall Plot作成中にエラー: {e}")
                return False
        else:
            print(f"警告: インデックス {idx} は処理済みデータに存在しません。")
            return False

    # 元の特徴量データを取得
    X_original = rice_data[X_processed.columns].copy() if X_processed.columns.tolist() else None
    if X_original is not None:
        # インデックスを合わせる
        X_original = X_original.loc[X_original.index.isin(X_processed.index)]

    # 上位3件のWaterfall Plotを作成
    for i, row in enumerate(top_3_instances, 1):
        idx = row['Index']
        country = row['Country']
        rank_info = f"Top {i} - Highest"
        create_extreme_waterfall_plot(idx, country, rank_info, X_processed, X_original, shap_values, explainer)

    # 下位3件のWaterfall Plotを作成
    for i, row in enumerate(bottom_3_instances, 1):
        idx = row['Index']
        country = row['Country']
        rank_info = f"Bottom {i} - Lowest"
        create_extreme_waterfall_plot(idx, country, rank_info, X_processed, X_original, shap_values, explainer)

    # Waterfall Plotをまとめた要約ページを作成
    plt.figure(figsize=(10, 10))
    plt.text(0.5, 0.98, 'Waterfall Plots Summary: Extreme Values', horizontalalignment='center', fontsize=16, fontweight='bold')
    plt.text(0.5, 0.95, 'Analysis of top 3 highest and top 3 lowest predicted yield differences', horizontalalignment='center', fontsize=12)

    # 上位3国リスト
    plt.text(0.05, 0.9, 'Top 3 Highest Predicted Yield Differences:', fontsize=11, fontweight='bold')
    for i, row in enumerate(top_3_instances, 1):
        plt.text(0.1, 0.88 - i*0.02, f"{i}. {row['Country']}: {row['PredictedValue']:.4f}", fontsize=10)

    # 下位3国リスト
    plt.text(0.05, 0.78, 'Top 3 Lowest Predicted Yield Differences:', fontsize=11, fontweight='bold')
    for i, row in enumerate(bottom_3_instances, 1):
        plt.text(0.1, 0.76 - i*0.02, f"{i}. {row['Country']}: {row['PredictedValue']:.4f}", fontsize=10)

    plt.text(0.05, 0.65, 'Each waterfall plot shows how feature values contributed to model prediction,\nwith both original (pre-imputation) and imputed values.', fontsize=10)
    plt.axis('off')
    plt.savefig(os.path.join('section4_predictions_analysis', 'extreme_values', 'Extreme_Values_Waterfall_Summary.png'))
    plt.close()

    print("\nすべての極値インスタンスのWaterfall Plotが作成されました。")
    print("要約ページを 'section4_predictions_analysis/extreme_values/Extreme_Values_Waterfall_Summary.png' に保存しました。")
else:
    print("インスタンスデータが見つかりません。Waterfall Plotは作成されません。")


SHAP値による解釈を開始...


NameError: name 'shap' is not defined

全てのサンプルのSHAP値を1つの図にまとめ、各変数の重要度を可視化

In [ ]:
# ============= ５．Beeswarm plotの作成 =============

# Beeswarm Plot
plt.figure(figsize=(12, 8))
shap.summary_plot(shap_values, X_processed, plot_type="dot", show=False)
plt.title('SHAP Beeswarm Plot')
plt.tight_layout()
plt.savefig(os.path.join('section5_shap_analysis', 'SHAP_Beeswarm.png'))
plt.close()

# 変数重要度（SHAP値の絶対値平均）
feature_importance = pd.DataFrame({
    'Feature': X_processed.columns,
    'Importance': np.abs(shap_values).mean(axis=0)
})
feature_importance = feature_importance.sort_values('Importance', ascending=False)

plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=feature_importance)
plt.title('Feature Importance Based on SHAP Values')
plt.tight_layout()
plt.savefig(os.path.join('section5_shap_analysis', 'SHAP_Feature_Importance.png'))
plt.close()

SHAP値を用いた任意の2つの特徴量の相互作用の強さの定量化と、その予測値への影響の可視化

In [ ]:
# ============= ６．相互作用の解析 =============

# 相互作用の分析
print("\n相互作用の分析を開始...")

# SHAP Interaction Values
interaction_values = explainer.shap_interaction_values(X_processed)

# 相互作用の平均絶対値を計算
interaction_strength = np.zeros((X_processed.shape[1], X_processed.shape[1]))
for i in range(X_processed.shape[1]):
    for j in range(X_processed.shape[1]):
        interaction_strength[i, j] = np.abs(interaction_values[:, i, j]).mean()

# 相互作用の強さ（対角成分を除く）
interaction_df = []
for i in range(X_processed.shape[1]):
    for j in range(i+1, X_processed.shape[1]):
        interaction_df.append({
            'Feature1': X_processed.columns[i],
            'Feature2': X_processed.columns[j],
            'Interaction_Strength': interaction_strength[i, j] + interaction_strength[j, i]
        })
interaction_df = pd.DataFrame(interaction_df)
interaction_df = interaction_df.sort_values('Interaction_Strength', ascending=False)

# 上位10の相互作用を可視化
plt.figure(figsize=(14, 8))
top_interactions = interaction_df.head(10)
sns.barplot(x='Interaction_Strength', y=top_interactions.apply(lambda x: f"{x['Feature1']} × {x['Feature2']}", axis=1), data=top_interactions)
plt.title('Top 10 Feature Interactions')
plt.tight_layout()
plt.savefig(os.path.join('section6_interaction_analysis', 'SHAP_Top_Interactions.png'))
plt.close()

# 上位3つの相互作用に対するPartial Dependence Plots
print("\n上位3つの相互作用のPartial Dependence Plotを作成...")

# PDPの計算と可視化のための関数
def create_pdp_interaction(model, X, feature1, feature2, grid_resolution=20):
    # 特徴量の範囲を決定
    f1_min, f1_max = X[feature1].min(), X[feature1].max()
    f2_min, f2_max = X[feature2].min(), X[feature2].max()

    # グリッドポイントを作成
    f1_grid = np.linspace(f1_min, f1_max, grid_resolution)
    f2_grid = np.linspace(f2_min, f2_max, grid_resolution)

    # PDPの計算
    pdp_values = np.zeros((grid_resolution, grid_resolution))

    for i, v1 in enumerate(f1_grid):
        for j, v2 in enumerate(f2_grid):
            X_temp = X.copy()
            X_temp[feature1] = v1
            X_temp[feature2] = v2
            # 予測を計算
            predictions = model.predict(X_temp)
            pdp_values[i, j] = predictions.mean()

    # 結果をプロット
    plt.figure(figsize=(10, 8))
    plt.contourf(f1_grid, f2_grid, pdp_values.T, levels=50, cmap='viridis')
    plt.colorbar(label='Predicted Yield Difference')
    plt.xlabel(feature1)
    plt.ylabel(feature2)
    plt.title(f'Partial Dependence Plot: {feature1} × {feature2}')
    plt.tight_layout()
    plt.savefig(os.path.join('section6_interaction_analysis', f'PDP_{feature1}_{feature2}.png'))
    plt.close()

# 上位3つの相互作用に対するPDPを作成
for i in range(min(3, len(interaction_df))):
    interaction = interaction_df.iloc[i]
    feature1 = interaction['Feature1']
    feature2 = interaction['Feature2']
    print(f"Creating PDP for interaction: {feature1} × {feature2}")

    try:
        create_pdp_interaction(final_rf_model, X_processed, feature1, feature2)
    except Exception as e:
        print(f"Error creating PDP for {feature1} × {feature2}: {e}")

print("\n相互作用の分析が完了しました。")

ある特徴量の変化によって、予測値がどう変わるか、の可視化

In [ ]:
# ============= ７．異なる特徴量による予測値の変化を調査 =============

# 重要な特徴量を取得
top_features = feature_importance.head(5)['Feature'].tolist()
print(f"\n重要度上位5つの特徴量: {top_features}")

# 各特徴量の値が変化した場合の予測値の変化を調査
def plot_single_feature_pdp(model, X, feature, grid_resolution=50):
    # 特徴量の範囲を決定
    f_min, f_max = X[feature].min(), X[feature].max()

    # グリッドポイントを作成
    f_grid = np.linspace(f_min, f_max, grid_resolution)

    # PDPの計算
    pdp_values = []

    for val in f_grid:
        X_temp = X.copy()
        X_temp[feature] = val
        # 予測を計算
        predictions = model.predict(X_temp)
        pdp_values.append(predictions.mean())

    # 結果をプロット
    plt.figure(figsize=(10, 6))
    plt.plot(f_grid, pdp_values, 'b-', linewidth=2)
    plt.xlabel(feature)
    plt.ylabel('Predicted Yield Difference')
    plt.title(f'Partial Dependence Plot: {feature}')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join('section7_feature_pdp', f'PDP_Single_{feature}.png'))
    plt.close()

# 上位5つの重要な特徴量に対するPDPを作成
for feature in top_features:
    print(f"Creating PDP for feature: {feature}")
    try:
        plot_single_feature_pdp(final_rf_model, X_processed, feature)
    except Exception as e:
        print(f"Error creating PDP for {feature}: {e}")

print("\n単一特徴量のPDPの作成が完了しました。")

print("\n分析が完了しました。結果をそれぞれのフォルダに保存しました。")

保存したフォルダを自分のPCにダウンロード

In [ ]:
import os
import zipfile
from google.colab import files as colab_files  # 名前を変更してインポート

def create_zip_and_download_fixed():
    """すべての分析結果フォルダをZIPアーカイブに圧縮してダウンロード（修正版）"""
    # 分析セクションのフォルダリスト
    sections = [
        "section1_data_preparation",
        "section2_model_comparison",
        "section3_final_model",
        "section4_predictions_analysis",
        "section5_shap_analysis",
        "section6_interaction_analysis",
        "section7_feature_pdp"
    ]

    # 圧縮ファイル名
    zip_filename = "biochar_analysis_results.zip"

    # すでに存在するZIPファイルを削除
    if os.path.exists(zip_filename):
        os.remove(zip_filename)

    # ZIPファイルを作成
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # 各フォルダ内のファイルを追加
        for section in sections:
            if os.path.exists(section):
                # フォルダ内のすべてのファイルとサブフォルダを取得
                for root, dirs, file_list in os.walk(section):
                    for file in file_list:
                        # ファイルへのフルパス
                        file_path = os.path.join(root, file)
                        # ZIPファイル内のパス (相対パス)
                        arcname = os.path.relpath(file_path)
                        # ZIPに追加
                        zipf.write(file_path, arcname)
                        print(f"Added to ZIP: {file_path}")

    # ZIPファイルをダウンロード (修正した部分)
    colab_files.download(zip_filename)  # colab_filesを使用
    print(f"\nダウンロード完了: {zip_filename}")
    print("すべての分析結果ファイルが含まれています。")

# 修正した関数を実行
create_zip_and_download_fixed()

Added to ZIP: section1_data_preparation/Yield_Difference_Distribution_After_Filtering.png
Added to ZIP: section1_data_preparation/histograms/hist_Texture_initial.png
Added to ZIP: section1_data_preparation/histograms/hist_N_biochar.png
Added to ZIP: section1_data_preparation/histograms/hist_SOC_initial.png
Added to ZIP: section1_data_preparation/histograms/hist_Temperature_before_imputation.png
Added to ZIP: section1_data_preparation/histograms/hist_Temperature.png
Added to ZIP: section1_data_preparation/histograms/hist_P_biochar.png
Added to ZIP: section1_data_preparation/histograms/hist_Texture_initial_before_imputation.png
Added to ZIP: section1_data_preparation/histograms/hist_Type_biochar.png
Added to ZIP: section1_data_preparation/histograms/hist_C_biochar.png
Added to ZIP: section1_data_preparation/histograms/hist_pH_biochar.png
Added to ZIP: section1_data_preparation/histograms/hist_PyrolysisTemperature_biochar_before_imputation.png
Added to ZIP: section1_data_preparation/histo

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


ダウンロード完了: biochar_analysis_results.zip
すべての分析結果ファイルが含まれています。
